In [ ]:
##SkySport

In [41]:
import time  
from selenium import webdriver  
from selenium.webdriver.edge.service import Service as EdgeService  
from selenium.webdriver.edge.options import Options as EdgeOptions  
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  

# 1️⃣ **Set up Microsoft Edge WebDriver**
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

# 2️⃣ **Navigate to your target URL**
url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

# Wait for the page to load
time.sleep(5)

try:
    load_more_button = driver.find_element(By.CSS_SELECTOR, "button.ui-liveblog-button--load-more")
    print("✅ Load More button found!")

    # Scroll into view
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
    time.sleep(2)  # Allow page to adjust

    # Force click using JavaScript
    driver.execute_script("arguments[0].click();", load_more_button)
    print("✅ Load More button clicked with JavaScript!")

except Exception as e:
    print(f"❌ Load More button NOT found or not clickable: {e}")

✅ Load More button found!
✅ Load More button clicked with JavaScript!


In [45]:
import time
import re
import pandas as pd
from datetime import datetime

# Selenium imports
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 1️⃣ **Set up Microsoft Edge WebDriver**
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
# **Disable headless mode to observe behavior**
# edge_options.add_argument("--headless")  # Remove this for visual debugging
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

# 2️⃣ **Navigate to your target URL**
url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

# 3️⃣ **Wait for JavaScript to load the initial page content**
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ncpost-list-post")))

# 4️⃣ **Click Load More Button Once**
try:
    load_more_button = driver.find_element(By.CLASS_NAME, "ui-liveblog-button--load-more")
    if load_more_button.is_displayed():
        print("✅ Load More button found! Clicking once for testing...")

        # Ensure button is in view
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        time.sleep(1)  # Allow scrolling effect
        
        # Click using JavaScript
        driver.execute_script("arguments[0].click();", load_more_button)

        # Allow time for new content to load
        time.sleep(5)  # Adjust based on your connection speed

        # Ensure new content appears before proceeding
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ncpost-list-post")))

        print("✅ Load More button clicked successfully!")

    else:
        print("❌ Load More button found but not visible.")
except Exception as e:
    print(f"❌ Load More button NOT found or not clickable: {e}")

# 5️⃣ **Get the full rendered HTML after clicking Load More**
html = driver.page_source

# 6️⃣ **Close the browser**
driver.quit()

# 7️⃣ **Parse HTML with BeautifulSoup**
soup = BeautifulSoup(html, "html.parser")

# 8️⃣ **Find all articles inside both the pinned and main list containers**
articles = soup.select("div.ncpost-list-post")

news_data = []

for article in articles:
    # **Extract Headline**
    headline_tag = article.find("h2", class_="ncpost-title")
    headline_text = headline_tag.get_text(strip=True) if headline_tag else "No Headline"

    # **Extract Date & Time from <time>**
    time_tag = article.find("time", class_="ncpost-timestamp")
    if time_tag and time_tag.has_attr("datetime"):
        datetime_str = time_tag["datetime"]
        date_obj = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%fZ")

        # Fix for Windows compatibility
        try:
            date_text = date_obj.strftime("%#m/%#d/%Y")  # Works on Windows
        except ValueError:
            date_text = date_obj.strftime("%-m/%-d/%Y")  # Works on Linux/macOS
            
        time_text = date_obj.strftime("%H:%M")  # Format as HH:MM
    else:
        date_text, time_text = "Unknown", "00:00"

    # **Extract Body Context (All Paragraphs)**
    paragraphs = article.select("div.ncpost-content p")
    body_text = " ".join(p.get_text(strip=True) for p in paragraphs) if paragraphs else "No Content"

    # **Append results**
    news_data.append([date_text, time_text, headline_text, body_text])

# 9️⃣ **Convert to DataFrame**
dfs = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "Body Context"])

# 🔟 **Save or Display the DataFrame**
print(dfs.head())
dfs.to_csv("skysports_liverpool_test.csv", index=False)

print("✅ Scraping completed successfully!")

✅ Load More button found! Clicking once for testing...
✅ Load More button clicked successfully!
        Date   Time                                           Headline  \
0  3/17/2025  14:33  Merson calls for perspective after 'bad week' ...   
1  3/17/2025  11:59  Ref Watch: Carabao Cup Final - Big decisions a...   
2  3/17/2025  11:08  Van Dijk: There'll be news on my Liverpool fut...   
3  3/16/2025  20:26           LISTEN: Carabao Cup final review podcast   
4  3/16/2025  19:56       Carra: Worst cup final performance for years   

                                        Body Context  
0  Paul Merson says Liverpool have had a bad week...  
1  The Ref Watch team cast their eye over the big...  
2  LiverpoolcaptainVirgil van Dijksays there will...  
3  Mark Chapman is joined by Shay Given, Les Ferd...  
4  Sky Sports' Jamie Carragher reflecting on Live...  
✅ Scraping completed successfully!


In [46]:
import time
import re
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 1️⃣ **Set up Microsoft Edge WebDriver**
edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
# Disable headless mode for debugging
# edge_options.add_argument("--headless")  # Remove this to see browser actions
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

# 2️⃣ **Navigate to the target URL**
url = "https://www.skysports.com/football/live-blog/11669/13025501/liverpool-transfer-news-rumours-and-gossip-live-updates-and-latest-on-deals-signings-loans-and-contracts"
driver.get(url)

# 3️⃣ **Wait for the page to fully load**
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ncpost-list-post")))

# 4️⃣ **Click "Load More" repeatedly until it disappears**
def click_load_more():
    while True:
        try:
            load_more_button = driver.find_element(By.CLASS_NAME, "ui-liveblog-button--load-more")
            
            if load_more_button.is_displayed():
                print("✅ Load More button found! Clicking...")

                # Ensure the button is in view
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
                time.sleep(1)  # Allow scrolling effect

                # Click using JavaScript (to avoid "not clickable" errors)
                driver.execute_script("arguments[0].click();", load_more_button)

                # Wait for new content to load
                time.sleep(5)

                # Ensure new elements are loaded before proceeding
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ncpost-list-post")))

            else:
                print("❌ Load More button no longer visible.")
                break
        
        except Exception as e:
            print(f"❌ Load More button NOT found or all content loaded: {e}")
            break  # Stop when button disappears

# Call function to load all news
click_load_more()

# 5️⃣ **Get the full rendered HTML after all content is loaded**
html = driver.page_source

# 6️⃣ **Close the browser**
driver.quit()

# 7️⃣ **Parse HTML with BeautifulSoup**
soup = BeautifulSoup(html, "html.parser")

# 8️⃣ **Find all articles inside both the pinned and main list containers**
articles = soup.select("div.ncpost-list-post")

news_data = []

for article in articles:
    # **Extract Headline**
    headline_tag = article.find("h2", class_="ncpost-title")
    headline_text = headline_tag.get_text(strip=True) if headline_tag else "No Headline"

    # **Extract Date & Time from <time>**
    time_tag = article.find("time", class_="ncpost-timestamp")
    if time_tag and time_tag.has_attr("datetime"):
        datetime_str = time_tag["datetime"]
        date_obj = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%fZ")

        # Fix for Windows compatibility
        try:
            date_text = date_obj.strftime("%#m/%#d/%Y")  # Works on Windows
        except ValueError:
            date_text = date_obj.strftime("%-m/%-d/%Y")  # Works on Linux/macOS
            
        time_text = date_obj.strftime("%H:%M")  # Format as HH:MM
    else:
        date_text, time_text = "Unknown", "00:00"

    # **Extract Body Context (All Paragraphs)**
    paragraphs = article.select("div.ncpost-content p")
    body_text = " ".join(p.get_text(strip=True) for p in paragraphs) if paragraphs else "No Content"

    # **Append results**
    news_data.append([date_text, time_text, headline_text, body_text])

# 9️⃣ **Convert to DataFrame**
dfs = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "Body Context"])

# 🔟 **Save or Display the DataFrame**
print(dfs.head())
dfs.to_csv("skysports_liverpool_full.csv", index=False)

print("✅ Scraping completed successfully!")

✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More button found! Clicking...
✅ Load More 